In [5]:
!pip install -r requirements.txt

# Load data

In [ ]:
do_train = False

In [6]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
import nltk
import numpy as np
nltk.download('punkt')
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import regex
from transformers import BertForSequenceClassification
import csv
data = pd.read_pickle('our_train.pkl')
test_df = pd.read_pickle('our_test.pkl')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# to convert authors into numbers
author_to_number = {
    'EAP': 0,
    'HPL': 1,
    'MWS': 2
    
}
test_df.head()
data.head()
# lowercase, removing punctuation and tookenize sentences. Converting labels to int
for i in range(len(data)):
    data['text'].iloc[i] = regex.sub(r'[^\w\s]', '',data['text'].iloc[i].lower())
    data['author'].iloc[i] = author_to_number[data['author'].iloc[i]]
data.sample(frac=1)

for i in range(len(test_df)):
    test_df['text'].iloc[i] = regex.sub(r'[^\w\s]', '',test_df['text'].iloc[i].lower())
    test_df['author'].iloc[i] = author_to_number[test_df['author'].iloc[i]]
test_df.sample(frac=1)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,text,author
18951,id14609,his madness or at least his partial perversion...,1
18866,id03122,from the east tempestuous winds arose and chil...,1
19297,id13713,he meditated on what was best to be done,2
18163,id16448,nex day obed an thutty two others was in gaol ...,1
19264,id20308,the bodies of fat and fleshy persons with smal...,0
...,...,...,...
18049,id24120,i saw not then what i now clearly perceive tha...,0
17863,id22288,we could not perceive its necessity,0
19185,id06199,condescend therefore to have patience with my ...,2
18421,id15204,arose in good health and spirits and was aston...,0


In [7]:
def split_train_val_test(df, props=[.9, .1]):
    assert round(sum(props), 2) == 1 and len(props) >= 2
    train_df, val_df = None, None
    
    train_size = int(props[0] * len(df))
    val_size =  train_size + int(props[1] * len(df))
    train_df = df.iloc[0:train_size]
    val_df = df.iloc[train_size:]
    return train_df, val_df


train_df, val_df = split_train_val_test(data)

    
    

In [8]:
# Load a pretrained Bert model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    train_df.text, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    val_df.text, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)
encoded_data_test = tokenizer.batch_encode_plus(
    test_df.text, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_df.author)
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(val_df.author.tolist())

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test_df.author.tolist())

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [9]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [10]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 5

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [11]:

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [12]:
from sklearn.metrics import f1_score, accuracy_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_fun(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return accuracy_score(labels_flat, preds_flat)


In [13]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
if do_train:
    for epoch in tqdm(range(1, epochs+1)):
        
        model.train()
        
        loss_train_total = 0

        progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
        for batch in progress_bar:

            model.zero_grad()
            
            batch = tuple(b.to(device) for b in batch)
            
            inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1],
                    'labels':         batch[2],
                    }       

            outputs = model(**inputs)
            
            loss = outputs[0]
            loss_train_total += loss.item()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()
            
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        _, predictions, true_vals = evaluate(dataloader_validation)
                
        file = open('bert.model', 'w+')    
        torch.save(model.state_dict(), f'bert.model')
            
        tqdm.write(f'\nEpoch {epoch}')
        
        loss_train_avg = loss_train_total/len(dataloader_train)            
        tqdm.write(f'Training loss: {loss_train_avg}')
        
        val_loss, predictions, true_vals = evaluate(dataloader_validation)
        val_acc = accuracy_fun( predictions, true_vals)
        val_f1 = f1_score_func(predictions, true_vals)
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score: {val_f1}')
        tqdm.write(f'ACC: {val_acc}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/3171 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.5840490701469147
Validation loss: 0.4252163918488382
F1 Score: 0.8477904634115037
ACC: 0.8479001135073779


Epoch 2:   0%|          | 0/3171 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.3464821837534997
Validation loss: 0.7633360702446833
F1 Score: 0.8393223142873977
ACC: 0.8388195232690124


...


Epoch 3
Training loss: 0.18119410437843494
Validation loss: 0.7062933288667801
F1 Score: 0.8704427472896402
ACC: 0.8706015891032917


Epoch 4:   0%|          | 0/3171 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.08554907753374494
Validation loss: 0.844409524556035
F1 Score: 0.8666019921471458
ACC: 0.8666288308740068


Epoch 5:   0%|          | 0/3171 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.04499511412640161
Validation loss: 0.8854696085203428
F1 Score: 0.8738652859782451
ACC: 0.8740068104426788


In [16]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('bert.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_test)
print(accuracy_fun(predictions, true_vals))
print(f1_score_func(predictions, true_vals))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

0.8758901322482198
0.8758414031500741
